In [1]:
name = "CombineSignals"
residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [3]:
alphas = [
    "UserItemBiases",
    "MatrixFactorization10",
    "MatrixFactorization20",
    "MatrixFactorization40",
    "ItemCF.16",
    "ItemCF.64",
    "ItemCF.256",
    "ItemCF.1024",
];

In [4]:
function evaluate(X, y, β)
    y_pred = X * β
    y_pred = clamp.(y_pred, 1, 10)
    rmse(y, y_pred), mae(y, y_pred), r2(y, y_pred), mse(y, y_pred)
end;

In [5]:
function get_dep(split)
    return get_split(split).rating
end

function get_indep(split, alphas)
    users = get_split(split).user
    X = zeros(length(users), length(alphas))
    for j = 1:length(alphas)
        X[:, j] = get_alpha(alphas[j], split).rating
    end
    X
end;

In [6]:
β = get_indep("validation", alphas) \ get_dep("validation")

8-element Vector{Float64}:
  0.9941878913040841
  0.1246171761629176
  0.14088659732696796
  0.1907372958624015
  0.16925025865287807
  0.019569427350873577
 -0.010596271592077086
  0.5123345337463652

In [14]:
evaluate(get_indep("validation", alphas), get_dep("validation"), β)

(1.1025944225767024, 0.7998772497361326, 0.6165618992434283, 1.2157144606972519)

In [8]:
evaluate(get_indep("test", alphas), get_dep("test"), β)

(1.1027292962622612, 0.7999783483304447, 0.6165369542674858, 1.216011900835062)

In [9]:
write_params(Dict("β" => β, "alphas" => alphas));